In [1]:
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

chat_model = ChatOllama(model="mistral")

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OllamaEmbeddings(model="mistral"),
)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
  ("system", "Answer the question based only on the following context: {context}"),
  ("human", "Question: {question}")
])

setup_and_retrieval = RunnableParallel({"context": retriever, "question": RunnablePassthrough()})

chain = setup_and_retrieval | prompt | chat_model | StrOutputParser()

chain.invoke("where did harrison work?")

' Harrison worked at Kensho, but this information is unrelated to the context about bears and honey.'

# Get a `graph`

In [4]:
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

# `Get the prompts`

In [5]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Answer the question based only on the following context: {context}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Question: {question}'))])]